Diagnostics for NR

In [36]:
import sys
sys.path.append("../")

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import gw_eccentricity
from gw_eccentricity import get_available_methods
from gw_eccentricity import measure_eccentricity
from gw_eccentricity.load_data import load_waveform
from gw_eccentricity.plot_settings import colorsDict, lstyles, lwidths, figWidthsTwoColDict, figHeightsDict, use_fancy_plotsettings

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import glob
import numpy as np
from tqdm import tqdm
from gw_eccentricity.load_data import load_lvcnr_hack

In [38]:
sxs_nr_waveforms = sorted(glob.glob("../data/ecc_waveforms/Non-Precessing/SXS/BBH_S*.h5"))
et_waveforms = sorted(glob.glob("../data/ecc_waveforms/Non-Precessing/ET/*.h5"))

In [39]:
all_nr_waveforms = sorted(np.append(sxs_nr_waveforms, et_waveforms))

In [40]:
# in sxs_nr_waveforms
problematic_cases = [8, # need to increase flow and width 
                     34, # Takes too long to load, did not load within tested time.
                     37, # loads too short waveform
                    ]

In [41]:
num_orbits_dict= {"BBH_SKS_q1_e09_D130_DoNotControlLEqualsOneShape_AdjustSphereCExtents_Res3.h5": 3,
                 "BBH_SKS_q1_e09_D130_SKS_DoNotControlLEqualsOneShape_Res3.h5": 3}

In [42]:
def MakeDiagnosticPlots(methods, nr_waveforms, num_orbits_dict):
    """loop through all NR waveforms, and make diagnostics plots 
         methods - list of methods to apply
         nr_waveforms - list of paths to NR waveforms to apply methods to
    """

    # make output objects, one per method
    pps = [PdfPages(f"NR-diagnostics-{method}.pdf") for method in methods]


    comparisons = PdfPages("NR_diagnostics-comparisons.pdf")

    for idx, waveform_path in enumerate(nr_waveforms):
        w = waveform_path.split("/")[-1]
        print(f"===== {idx}/{len(nr_waveforms)} ===== {w} =====", flush=True)
        kwargs = {"filepath": waveform_path,
                "include_zero_ecc": True,
                "num_orbits_to_remove_as_junk": num_orbits_dict.get(w, 3)}
        dataDict = load_lvcnr_hack(**kwargs)
        tref_vec = dataDict["t"]


        # for ecc-comparison plot
        fig0, ax0 = plt.subplots(figsize=(12, 6))

        for method,pp in zip(methods, pps):
            tref_out, ecc_vec, mean_ano_vec, eccMethod = measure_eccentricity(
                tref_in=tref_vec,
                dataDict=dataDict,
                method=method, 
                return_gwecc_object=True,
                extra_kwargs={"debug": True,
                             "fits_kwargs": {"use_extra_checks": True}})
            fig, ax = eccMethod.make_diagnostic_plots(twocol=True)
            q = dataDict["params_dict"]["q"]
            ecc = dataDict["params_dict"]["ecc"]
            chi1 = np.around(dataDict["params_dict"]["chi1"], 5)
            chi2 = np.around(dataDict["params_dict"]["chi2"], 5)
            title = (f"case {idx}/{len(sxs_nr_waveforms)-1}, "
                    f"$q$={q:.1f}, "
                    f"$\chi_{1}$={chi1}, "
                    f"$\chi_{2}$={chi2}, "
                    f"ecc={ecc:.2f}\n"
                    f"method={method}, "
                    f"id={w}")
            fig.suptitle(title, ha="center", fontsize=10)
            fig.set_tight_layout(True)
            fig.savefig(pp,format='pdf')
            plt.close(fig)

            eccMethod.plot_measured_ecc(fig0, ax0, add_vline_at_tref=True if method == methods[0] else False, **{"label": method, "lw": lwidths[method], "ls": lstyles[method], "c": colorsDict[method]})
        ax0.set_title(f"q={q:.1f}, ecc={ecc:.2f}, {w}", fontsize=14)
        ax0.legend();
        fig0.savefig(comparisons,format='pdf')
        plt.close(fig0)

    for pp in pps:
        pp.close()
    comparisons.close()

In [43]:
# to test, analyse only two waveforms:
MakeDiagnosticPlots(["FrequencyFits"], 
                    [
                    "../data/ecc_waveforms/Non-Precessing/SXS/BBH_SHK_q8_e01_D14.5_Res3.h5",
                    # "../data/ecc_waveforms/Non-Precessing/SXS/BBH_SHK_q6_e095_D65_Res3.h5"
                    ],
                    #sxs_nr_waveforms[30:32], 
                    num_orbits_dict)

===== 0/1 ===== BBH_SHK_q8_e01_D14.5_Res3.h5 =====
t_analyse[0]=-7481.588149863342, t_analyse[-1]=-0.08814983612393879, global fit to t<=-4239.188149851546
global fit: guess p0=[0.03899498558174935, 3.9090489904631155e-06, 0],  t_center=-3740.838149849733
            bounds=[[0.0, 0.0, -0.07051986889915103], [0.38994985581749353, 0.00010424130641234975, 3740.838149849733]]
            result p_global=[ 4.46061585e-02  3.88388104e-06 -7.05198689e-02]
=== count=1 ============================================================
FindExtremaNearIdxRef  idx_ref=7627, K_initial=1.100, p_initial=0.65(-0.07-t)^-0.326, refine_extrema=False
       find_peaks: distance=2503, prominence=3.753199177844828e-05
it=1:  [0 / 7627 / 22771],  K=1.100
       idx_extrema=   [ 1113  5622 10061 14415 18683], Nleft=2, Nright=3
       t[idx_extrema]=[-7370.29 -6919.39 -6475.49 -6040.09 -5613.29]
       Count_Nright_short=1
       idx_ref increased to 12238
       idx_hi increased to 25209
it=2:  [0 / 12238 / 25209]

In [8]:
# ALL SXS-WAVEFORMS.  THIS TAKES A FEW MINUTES !!!
MakeDiagnosticPlots(gw_eccentricity.get_available_methods(), sxs_nr_waveforms, num_orbits_dict)

===== 0/61 ===== BBH_SHK_q10_e01_D14.5_Res3.h5 =====


/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:450: UserWarning: Encountered negative eccentricity.
  warnings.warn("Encountered negative eccentricity.")


===== 1/61 ===== BBH_SHK_q10_e01_D16_Res3.h5 =====


/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:450: UserWarning: Encountered negative eccentricity.
  warnings.warn("Encountered negative eccentricity.")


===== 2/61 ===== BBH_SHK_q10_e05_D28_Res3.h5 =====
===== 3/61 ===== BBH_SHK_q10_e05_D30_Res3.h5 =====
===== 4/61 ===== BBH_SHK_q10_e07_D45_Res3.h5 =====
===== 5/61 ===== BBH_SHK_q1_0_0_e01_D16_FastDyn_Res3.h5 =====


...

/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:643: UserWarning: Number of pericenters and number of apocenters differ by 2. This implies that pericenters and apocenters are not appearing alternately.
  warnings.warn(
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:590: UserWarning: At least a pair of apocenters are too close. Minimum orbital phase diff is 0.22959956968837147. Times of occurrences are [-9601.72655884]
  warnings.warn(f"At least a pair of {extrema_type} are too close."
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:596: UserWarning: Phase shift closer to pi than 2 pi detected.
  warnings.warn("Phase shift closer to pi than 2 pi detected.")
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:608: UserWarning: At least a pair of apocenters are too far. Maximum orbital phase diff is 8.534371615420554. Times of occurrences are [-8986.92655883]
  warnings.warn(f"At least

===== 59/61 ===== BBH_SKS_q1_e09_D130_DoNotControlLEqualsOneShape_AdjustSphereCExtents_Res3.h5 =====


/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:643: UserWarning: Number of pericenters and number of apocenters differ by 2. This implies that pericenters and apocenters are not appearing alternately.
  warnings.warn(
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:590: UserWarning: At least a pair of apocenters are too close. Minimum orbital phase diff is 0.037841946546084415. Times of occurrences are [-6717.42448362]
  warnings.warn(f"At least a pair of {extrema_type} are too close."
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:596: UserWarning: Phase shift closer to pi than 2 pi detected.
  warnings.warn("Phase shift closer to pi than 2 pi detected.")
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:608: UserWarning: At least a pair of apocenters are too far. Maximum orbital phase diff is 10.795491125821371. Times of occurrences are [-5445.52448361]
  warnings.warn(f"At lea

===== 60/61 ===== BBH_SKS_q1_e09_D130_SKS_DoNotControlLEqualsOneShape_Res3.h5 =====


/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:685: UserWarning: There is at least one instance where pericenters and apocenters do not appear alternately.
  warnings.warn(
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:596: UserWarning: Phase shift closer to pi than 2 pi detected.
  warnings.warn("Phase shift closer to pi than 2 pi detected.")
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:608: UserWarning: At least a pair of pericenters are too far. Maximum orbital phase diff is 10.386346482137355. Times of occurrences are [-6100.49718692]
  warnings.warn(f"At least a pair of {extrema_type} are too far."
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:685: UserWarning: There is at least one instance where pericenters and apocenters do not appear alternately.
  warnings.warn(
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:590: UserWarning: At least

### Compare ecc vs time plot for different methods with varying num orbits before merger

In [8]:
def MakeDiagnosticPlotsAsFunctionOfNumOrbits(methods, nr_waveforms, list_num_orbits_to_exclude_before_merger, style="APS"):
    """loop through all NR waveforms, and make diagnostics plots.
    
    For different methods by varying the num orbits before merger variable.
    
    Parameters:
    -----------
    methods:
        list of methods to apply.
        
    nr_waveforms:
        list of paths to NR waveforms to apply methods to.
        
    list_num_orbits_to_exclude_before_merger:
        list of num orbits to exclude before mergers.
    
    style:
        Settings for plots.
    """

    # make output object
    num_strings = "_".join([f"{x:.1f}" for x in list_num_orbits_to_exclude_before_merger])
    pp = PdfPages(f"NR-diagnostics-for-num-orbits_{num_strings}.pdf")

    for idx, waveform_path in enumerate(nr_waveforms):
        w = waveform_path.split("/")[-1]
        print(f"===== {idx}/{len(nr_waveforms)} ===== {w} =====", flush=True)
        kwargs = {"filepath": waveform_path,
                "include_zero_ecc": True,
                "num_orbits_to_remove_as_junk": num_orbits_dict.get(w, 3)}
        dataDict = load_lvcnr_hack(**kwargs)
        tref_vec = dataDict["t"]


        # for ecc-comparison plot
        nrows = len(list_num_orbits_to_exclude_before_merger)
        use_fancy_plotsettings(style=style)
        fig, axes = plt.subplots(nrows=nrows, figsize=(figWidthsTwoColDict[style], nrows * figHeightsDict[style]),
                                sharex=True)
        if nrows == 1:
            axes = [axes]
    
        for idx1, num_orbits in enumerate(list_num_orbits_to_exclude_before_merger):
            for idx2, method in enumerate(methods):
                try:
                    tref_out, ecc_vec, mean_ano_vec, gwecc_obj = measure_eccentricity(
                        tref_in=tref_vec,
                        dataDict=dataDict,
                        method=method, 
                        return_gwecc_object=True,
                        extra_kwargs={"debug": False,
                                     "num_orbits_to_exclude_before_merger": num_orbits})
                    gwecc_obj.plot_measured_ecc(fig, axes[idx1], add_vline_at_tref=True if idx2 == 0 else False,
                                                **{"label": method,
                                                   "c": colorsDict[method],
                                                   "ls": lstyles[method],
                                                   "lw": lwidths[method]})
                except Exception:
                    print(f"{method} Fails for {w} with {num_orbits}.")
            axes[idx1].text(0.99, 0.95, f"number of orbits = {num_orbits:.1f}", transform=axes[idx1].transAxes,
                           ha="right", va="top", fontsize=10)
            axes[idx1].set_xlabel("")
        axes[0].legend(labelspacing=0.2, handlelength=1, fontsize=10)
        axes[-1].set_xlabel(r"$t$ [$M$]")
        q = dataDict["params_dict"]["q"]
        ecc = dataDict["params_dict"]["ecc"]
        chi1 = np.around(dataDict["params_dict"]["chi1"], 5)
        chi2 = np.around(dataDict["params_dict"]["chi2"], 5)
        title = (f"case {idx}/{len(sxs_nr_waveforms)-1}, "
                f"$q$={q:.1f}, "
                f"$\chi_{1}$={chi1}, "
                f"$\chi_{2}$={chi2}, "
                f"ecc={ecc:.2f}\n"
                f"id={w}")
        axes[0].set_title(title, ha="center", fontsize=10)
        fig.set_tight_layout(True)
        fig.savefig(pp,format='pdf')
        plt.close(fig)
    pp.close()

In [9]:
MakeDiagnosticPlotsAsFunctionOfNumOrbits(get_available_methods(), sxs_nr_waveforms, [1, 2, 3, 4])

===== 0/61 ===== BBH_SHK_q10_e01_D14.5_Res3.h5 =====


/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:465: UserWarning: Encountered negative eccentricity.
  warnings.warn("Encountered negative eccentricity.")


===== 1/61 ===== BBH_SHK_q10_e01_D16_Res3.h5 =====


/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:465: UserWarning: Encountered negative eccentricity.
  warnings.warn("Encountered negative eccentricity.")


===== 2/61 ===== BBH_SHK_q10_e05_D28_Res3.h5 =====
===== 3/61 ===== BBH_SHK_q10_e05_D30_Res3.h5 =====
===== 4/61 ===== BBH_SHK_q10_e07_D45_Res3.h5 =====
===== 5/61 ===== BBH_SHK_q1_0_0_e01_D16_FastDyn_Res3.h5 =====


...

/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:662: UserWarning: Number of pericenters and number of apocenters differ by 2. This implies that pericenters and apocenters are not appearing alternately.
  warnings.warn(
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:608: UserWarning: At least a pair of apocenters are too close. Minimum orbital phase diff is 0.22959956968837147. Times of occurrences are [-9601.72655884]
  warnings.warn(f"At least a pair of {extrema_type} are too close."
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:614: UserWarning: Phase shift closer to pi than 2 pi detected.
  warnings.warn("Phase shift closer to pi than 2 pi detected.")
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:626: UserWarning: At least a pair of apocenters are too far. Maximum orbital phase diff is 8.534371615420554. Times of occurrences are [-8986.92655883]
  warnings.warn(f"At least

===== 59/61 ===== BBH_SKS_q1_e09_D130_DoNotControlLEqualsOneShape_AdjustSphereCExtents_Res3.h5 =====


/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:662: UserWarning: Number of pericenters and number of apocenters differ by 2. This implies that pericenters and apocenters are not appearing alternately.
  warnings.warn(
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:608: UserWarning: At least a pair of apocenters are too close. Minimum orbital phase diff is 0.037841946546084415. Times of occurrences are [-6717.42448362]
  warnings.warn(f"At least a pair of {extrema_type} are too close."
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:614: UserWarning: Phase shift closer to pi than 2 pi detected.
  warnings.warn("Phase shift closer to pi than 2 pi detected.")
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:626: UserWarning: At least a pair of apocenters are too far. Maximum orbital phase diff is 10.795491125821371. Times of occurrences are [-5445.52448361]
  warnings.warn(f"At lea

===== 60/61 ===== BBH_SKS_q1_e09_D130_SKS_DoNotControlLEqualsOneShape_Res3.h5 =====


/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:706: UserWarning: There is at least one instance where pericenters and apocenters do not appear alternately.
  warnings.warn(
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:614: UserWarning: Phase shift closer to pi than 2 pi detected.
  warnings.warn("Phase shift closer to pi than 2 pi detected.")
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:626: UserWarning: At least a pair of pericenters are too far. Maximum orbital phase diff is 10.386346482137355. Times of occurrences are [-6100.49718692]
  warnings.warn(f"At least a pair of {extrema_type} are too far."
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:706: UserWarning: There is at least one instance where pericenters and apocenters do not appear alternately.
  warnings.warn(
/home1/md.shaikh/Eccentricity/notebook/../gw_eccentricity/eccDefinition.py:608: UserWarning: At least